In [1]:
#piplite presence will be shown with these statememnts
try:    
    import piplite
except ModuleNotFoundError:
    piplite=None
# Use these packages for download of data
if piplite:
    await piplite.install("mplfinance")
    await piplite.install("finta")
    await piplite.install("ipympl")
    await piplite.install("ipywidgets")
    try:
        await piplite.install("/pypi/backtrader-1.9.76.123-py3-none-any.whl")
    except ValueError as e:
        print("localExampleEnv-failed")
        try: 
            await piplite.install("http://localhost/staticcollected/_output/pypi/backtrader-1.9.76.123-py3-none-any.whl")
        except ValueError as e:
            print("localProd00env-failed")
            await piplite.install("http://www.stockwhiz.in/staticcollected/_output/pypi/backtrader-1.9.76.123-py3-none-any.whl")
    # This is for the fetch in piplite
    import js 
    from pyodide.ffi import to_js
    from js import Object
else:
    import yfinance
#3 Imports of all the files
import json
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import mplfinance as mpf
import scipy as sp
from IPython.display import display, Markdown
import backtrader as bt
import finta
import ipywidgets as widgets

%matplotlib widget
pd.options.display.max_columns = None

---

# 05 MACD Confirmed RSI

Click $>>$ button to run the entire script first

**Buy Signal**
> BUY 100 shares when macd(7,13,9) higher than macd signal(7,13,9) and 25 rsi crosses above 30 at day candle interval using Heikin-Ashi Chart . Enter trade between 09:00 to 23:30

**Sell Signal**
> SELL 100 shares when macd(7,13,9) lower than macd signal(7,13,9) and 25 rsi crosses above 71 or at stop loss % of 15.0 or target profit % of 35.0 at day candle interval using Heikin-Ashi Chart chart.

Please feel free to change to any **NSE SYMBOL** to the below list, separated by a comma

In [2]:
symbols_1 = "GUJGASLTD"

In [3]:
corsprox = "https://corsproxy.io/?"
apiBaseURL = "https://query1.finance.yahoo.com/v8/finance/chart/"
rangeOfData = "6mo"
intervalOfData = "1d"
apiSymbol = []
for symbol in symbols_1.split(","):
    apiSymbol.append(symbol+".NS")
stockPandasTot_1 = {}
for apiSymbolIndivi in apiSymbol:
    print(apiSymbolIndivi)
    apiCompleteURL = corsprox+apiBaseURL+apiSymbolIndivi+\
        "?range="+rangeOfData+"&interval="+intervalOfData
    options = { "mode": "cors",
                "credentials":"omit",
                "headers": {'Accept': 'application/json',
                            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
                            'Origin':'https://yahoo.com',
                            }
    }
    if piplite:
        resp = await js.fetch(apiCompleteURL, to_js(options, dict_converter=Object.fromEntries))
        text = await resp.text()
        pan = pd.read_json(text)
        a_hist = pan['chart'][1][0]
        stockPandas = pd.DataFrame(a_hist['indicators']['quote'][0],
                               index=pd.to_datetime(a_hist['timestamp'],unit='s'))
        stockPandas = stockPandas.reindex(columns=['open','high','low','close','volume'])
        stockPandas = stockPandas[::-1]
    else:
        stockPandas = yfinance.download(apiSymbolIndivi,period=rangeOfData,interval=intervalOfData,progress=False)
        stockPandas = stockPandas.round(2).rename(columns={
                                                    'Open': 'open',
                                                    'High': 'high',
                                                    'Low': 'low',
                                                    'Close': 'close',
                                                    'Adj Close': 'adj close',
                                                    'Volume': 'volume'
                                                })
        stockPandas = stockPandas[::-1]
    stockPandasTot_1[apiSymbolIndivi] = stockPandas

GUJGASLTD.NS


## An MACD line

<img src="https://bpcdn.co/images/2015/03/25135506/MACD.png" width="700" height="300"/>

### Buy signal
> BUY 100 shares when macd(7,13,9) higher than macd signal(7,13,9) and 25 rsi crosses above 30 at day candle interval using Heikin-Ashi Chart. Enter trade between 09:00 to 23:30
- if $macd.macd[0] - macd.signal[0] > 0$
    - The crossover happens today
        - $rsi[-1] < 35$
        - $rsi[0] > 35$
    
### Sell signal
> SELL 100 shares when macd(7,13,9) lower than macd signal(7,13,9) and 25 rsi crosses above 71 or at stop loss % of 15.0 or target profit % of 35.0 at day candle interval using Heikin-Ashi Chart chart.

If either of the below conditions are met
- Signal condition met if $macd.macd[0] - macd.signal[0] < 0$
    - if rsi crossover happens today
        - $rsi[-1] < 75 $
        - $rsi[0] > 75 $
- Stop Loss condition met if $stock.close[0] < order.buy*0.85$
- Target price condition met if $stock.close[0] > order.buy*1.35$

In [4]:
display(Markdown("# Set the variables to this strategy here"))
display(Markdown("> We will improve UI shortly"))
display(Markdown("If interested on the `coding` aspect, please click the 3 dot's above to see comments on code and change accordingly"))
@widgets.interact(shortPeriod=widgets.IntSlider(min=2, max=15, step=1, value=7,continuous_update=False),longPeriod=widgets.IntSlider(min=9, max=30, step=1, value=13,continuous_update=False),\
                  signalPeriod=widgets.IntSlider(min=5, max=15, step=1, value=9,continuous_update=False),rsiPeriod=widgets.IntSlider(min=9, max=30, step=1, value=25,continuous_update=False),\
                  rsiBuyValue=widgets.IntSlider(min=20, max=50, step=1, value=30,continuous_update=False),rsiSellValue=widgets.IntSlider(min=50, max=80, step=1, value=71,continuous_update=False),\
                  numberOfStocks=widgets.IntSlider(min=0, max=30, step=1, value=10,continuous_update=False),initialCash=widgets.IntSlider(min=0, max=10000, step=1, value=5000,continuous_update=False),
                 )
def voilas(shortPeriod,longPeriod,signalPeriod,rsiPeriod,rsiBuyValue,rsiSellValue,numberOfStocks,initialCash):
    class CrossOverStrat(bt.Strategy):
        def __init__(self):
            # Keep a reference to the "close" line in the data[0] dataseries
            self.i =0
            self.dataclose = self.datas[0].close
            self.heikinDate = bt.indicators.HeikinAshi(self.datas[0])
            #Some indicators
            self.macd = bt.indicators.MACDHisto(self.heikinDate,period_me1=shortPeriod,period_me2=longPeriod,period_signal=signalPeriod)
            self.rsi = bt.indicators.RSI_SMA(self.heikinDate,period=rsiPeriod)

            # To keep track of pending orders and buy price/commission
            self.order = None
            self.buyprice = None
            self.buycomm = None

        def log(self, txt, dt=None):
            ''' Logging function for this strategy'''
            dt = dt or self.datas[0].datetime.date(0)
            print('%s, %s' % (dt.isoformat(), txt))

        def notify_order(self, order):
            if order.status in [order.Submitted, order.Accepted]:
                # Buy/Sell order submitted/accepted to/by broker - Nothing to do
                return

            # Check if an order has been completed
            # Attention: broker could reject order if not enough cash
            if order.status in [order.Completed]:
                if order.isbuy():
                    self.log(
                        'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                        (order.executed.price,
                         order.executed.value,
                         order.executed.comm))

                    self.buyprice = order.executed.price
                    self.buycomm = order.executed.comm
                else:  # Sell
                    self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                             (order.executed.price,
                              order.executed.value,
                              order.executed.comm))

                self.bar_executed = len(self)

            elif order.status in [order.Canceled, order.Margin, order.Rejected]:
                self.log('Order Canceled/Margin/Rejected')

            self.order = None

        def notify_trade(self, trade):
            if not trade.isclosed:
                return

            self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                     (trade.pnl, trade.pnlcomm))

        def next(self):
            # Simply log the closing price of the series from the reference
            # Check if an order is pending ... if yes, we cannot send a 2nd one
            if self.order:
                return
            # Check if we are in the market
            if not self.position:
                if self.macd.histo[0] > 0:
                    if self.rsi[-1] < rsiBuyValue:
                        if self.rsi[0] > rsiBuyValue:
                            # Keep track of the created order to avoid a 2nd order
                            self.order = self.buy(size=numberOfStocks)
            else:
                # Already in the market ... we might sell
                if self.macd.histo[0] < 0:
                    if self.rsi[0] > rsiSellValue:
                        # Keep track of the created order to avoid a 2nd order
                        self.log(f'target signal reached')
                        self.order = self.sell(size=numberOfStocks)
                if self.dataclose[0] >= 1.35*self.buyprice:
                    self.log(f'target price reached')
                    self.order = self.sell(size=numberOfStocks)
                if self.dataclose[0] <= 0.85*self.buyprice:
                    self.log(f'stop loss reached')
                    self.order = self.sell(size=numberOfStocks)
    cerebro = bt.Cerebro()

    for symbol in stockPandasTot_1:
        data = bt.feeds.PandasData(dataname=stockPandasTot_1[symbol][::-1])
        cerebro.adddata(data)

    cerebro.addstrategy(CrossOverStrat)

    cerebro.broker.setcash(initialCash)

    cerebro.broker.setcommission(commission=0.001)

    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    cerebro.run()

    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())
    matplotlib.use("nbAgg")
    cerebro.plot(style='candlestick',iplot=True)

# Set the variables to this strategy here

> We will improve UI shortly

If interested on the `coding` aspect, please click the 3 dot's above to see comments on code and change accordingly

interactive(children=(IntSlider(value=7, continuous_update=False, description='shortPeriod', max=15, min=2), I…

---
### Understanding above plot

> Please increase the size of the plot to the entire width of the screen, before reading this section. You can do the same by dragging the triangular element shown at the bottom right corner of the plot

The plot above has quite a lines and legends on them.
The legend is available on the left side indicating various items.

We shall move from the bottom most element and move upwards.
#### MACD
The Moving Average Convergence/Divergence represents a variation in the average price of stocks, It consists of 2 lines representing a short period and a long period. A histogram represents the difference in values of these two lines. I would direct the user towards this [wiki](https://en.wikipedia.org/wiki/MACD)
#### RSI_SMA
This represent the rsi value of the stock for the said time period, as a slightly involved definition I direct the user towards [wiki](https://en.wikipedia.org/wiki/Relative_strength_index). These idicators are even used in economic decision making of countries.
#### Volume
The volume is indicated by bars of either gray or red colour. The respective value is availabe on the left part of the plot along the y-axis. Exact values for volume cannot be extracted currently. A red bar indicates a decrease in stock price over the time period. A grey bar indicates an increase in stock price over the given time period.
#### Close price line
The black line represents the close price line of the stock over a period of 48months, dates are visible at bottom of graph extracted from x value.

 *If you would like a candle stick pattern please add `style="candlestick"` to the arguments of the arguments of the plot command*
 
The default candlestick patterns are available over the course of the day, these follow the typical format with grey representing a rise in prices and red representing a fall in prices. Exact price and date values of cursor position are available at the bottom. The x-value represents date, while the y-value represents the price at cursor location.
#### Triangles
The triangles represent positions taken up according to the strategy mentioned by code above. A buy position/ signal will be generated if the triangle is green and pointed upwards, while a sell position/ sell signal is generated with the red downward pointing triangle.
#### Trading dots
The next canvas involves a set of dots representing either profit or losses which have occured by following the strategy. Here too the magnitudes can be separated by the colour of the dot and the cursor value on hovering over dots. One can also approximate the values with the data on the scale located on the right.
#### Portfolio value
The final screen on the top indicates the portfolio value and cash value. Each are differentiated by the respective color.

---
## Filtering Several Stocks

In this section we will be filtering through lists of stocks to understand which satisfy either our buy or sell signals.

We recommend the user to rebalance their portfolio by running this script once every day at 15:00 and rebalance their portfolio using the signals generated here.

### Legend

- $[-30,-1]$: Used to signify a sell signal
- $[1,30]$  : Used to signify a BUY signal
- $0$       : Used to signify no action required.

In [5]:
symbols_02 = "ADANIENT,ADANIPORTS,APOLLOHOSP,ASIANPAINT,AXISBANK,\
BAJAJ-AUTO,BAJFINANCE,BAJAJFINSV,BHARTIARTL,BPCL,BRITANNIA,\
CIPLA,COALINDIA,\
DIVISLAB,DRREDDY,\
EICHERMOT,\
GRASIM,\
HCLTECH,HDFC,HDFCBANK,HDFCLIFE,HEROMOTOCO,HINDALCO,HINDUNILVR,\
ICICIBANK,INDUSINDBK,INFY,ITC,\
JSWSTEEL,\
KOTAKBANK,\
LT,\
M&M,MARUTI,\
NESTLEIND,NTPC,\
ONGC,\
POWERGRID,\
RELIANCE,\
SBIN,SBILIFE,SUNPHARMA,\
TATAMOTORS,TATASTEEL,TCS,TATACONSUM,TECHM,TITAN,\
ULTRACEMCO,UPL,\
WIPRO"

In [6]:
corsprox = "https://corsproxy.io/?"
apiBaseURL = "https://query1.finance.yahoo.com/v8/finance/chart/"
rangeOfData = "3mo"
intervalOfData = "1d"
apiSymbol = []
for symbol in symbols_02.split(","):
    apiSymbol.append(symbol+".NS")
stockPandasTot_02 = {}
for apiSymbolIndivi in apiSymbol:
    print(apiSymbolIndivi)
    apiCompleteURL = corsprox+apiBaseURL+apiSymbolIndivi+\
        "?range="+rangeOfData+"&interval="+intervalOfData
    options = { "mode": "cors",
                "credentials":"omit",
                "headers": {'Accept': 'application/json',
                            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
                            'Origin':'https://yahoo.com',
                            }
    }
    if piplite:
        resp = await js.fetch(apiCompleteURL, to_js(options, dict_converter=Object.fromEntries))
        text = await resp.text()
        pan = pd.read_json(text)
        a_hist = pan['chart'][1][0]
        stockPandas = pd.DataFrame(a_hist['indicators']['quote'][0],
                               index=pd.to_datetime(a_hist['timestamp'],unit='s'))
        stockPandas = stockPandas.reindex(columns=['open','high','low','close','volume'])
        stockPandas = stockPandas[::-1]
    else:
        stockPandas = yfinance.download(apiSymbolIndivi,period=rangeOfData,interval=intervalOfData,progress=False)
        stockPandas = stockPandas.round(2).rename(columns={
                                                    'Open': 'open',
                                                    'High': 'high',
                                                    'Low': 'low',
                                                    'Close': 'close',
                                                    'Adj Close': 'adj close',
                                                    'Volume': 'volume'
                                                })
        stockPandas = stockPandas[::-1]
    stockPandasTot_02[apiSymbolIndivi] = stockPandas

ADANIENT.NS
ADANIPORTS.NS
APOLLOHOSP.NS
ASIANPAINT.NS
AXISBANK.NS
BAJAJ-AUTO.NS
BAJFINANCE.NS
BAJAJFINSV.NS
BHARTIARTL.NS
BPCL.NS
BRITANNIA.NS
CIPLA.NS
COALINDIA.NS
DIVISLAB.NS
DRREDDY.NS
EICHERMOT.NS
GRASIM.NS
HCLTECH.NS
HDFC.NS
HDFCBANK.NS
HDFCLIFE.NS
HEROMOTOCO.NS
HINDALCO.NS
HINDUNILVR.NS
ICICIBANK.NS
INDUSINDBK.NS
INFY.NS
ITC.NS
JSWSTEEL.NS
KOTAKBANK.NS
LT.NS
M&M.NS
MARUTI.NS
NESTLEIND.NS
NTPC.NS
ONGC.NS
POWERGRID.NS
RELIANCE.NS
SBIN.NS
SBILIFE.NS
SUNPHARMA.NS
TATAMOTORS.NS
TATASTEEL.NS
TCS.NS
TATACONSUM.NS
TECHM.NS
TITAN.NS
ULTRACEMCO.NS
UPL.NS
WIPRO.NS


In [7]:
## Variables
# Number of days checked [Can we push this dates checked to different cell?]
datesChecked = 16
# weights of each individual day (Shape in reshape function should be matched. )
weights = np.array([15, 10, 5, 2])
# Adding additional weights, will be helfull only while checking.
weights = np.append(weights, np.ones((datesChecked-4,1)))
signalBullDict = {}

In [17]:
for symbol_02 in stockPandasTot_02:
    # Reversing data because finta likes calculations with oldest date on top
    selectData = stockPandasTot_02[symbol_02][::-1]
    # These data are reversed again to show the latest date at the top row.
    MACD = finta.TA.MACD(selectData,7,13,9)[::-1]
    RSI = finta.TA.RSI(selectData,25)[::-1]
    diff = (MACD['MACD']- MACD['SIGNAL']).fillna(0)
print(symbol_02)
print((RSI >= 40) & (RSI.shift(1).fillna(50) < 40) )


WIPRO.NS
Date
2023-05-05    False
2023-05-04    False
2023-05-03    False
2023-05-02    False
2023-04-28    False
2023-04-27    False
2023-04-26    False
2023-04-25    False
2023-04-24    False
2023-04-21    False
2023-04-20    False
2023-04-19    False
2023-04-18     True
2023-04-17    False
2023-04-13    False
2023-04-12     True
2023-04-11    False
2023-04-10    False
2023-04-06    False
2023-04-05    False
2023-04-03    False
2023-03-31    False
2023-03-29    False
2023-03-28    False
2023-03-27    False
2023-03-24    False
2023-03-23    False
2023-03-22    False
2023-03-21    False
2023-03-20    False
2023-03-17    False
2023-03-16    False
2023-03-15    False
2023-03-14    False
2023-03-13    False
2023-03-10    False
2023-03-09    False
2023-03-08     True
2023-03-06    False
2023-03-03    False
2023-03-02    False
2023-03-01    False
2023-02-28    False
2023-02-27    False
2023-02-24    False
2023-02-23    False
2023-02-22    False
2023-02-21     True
2023-02-20    False
2023-0